In [1]:
import requests, os, logging
from google.cloud import bigquery
from google.oauth2 import service_account
import google
from unidecode import unidecode

import functions_framework
import google.cloud.logging

# BOOST_TOKEN = os.environ['BOOST_TOKEN']
BOOST_TOKEN = "***REMOVED***"
BOOST_TABLE = "offre_emploi"
BOOST_URL = f"https://www.province-sud.nc/drhouseweb/api/GOUV_WEB/societe/{BOOST_TABLE}/data?apiKey={BOOST_TOKEN}"

# PROD
GCP_PROJECT_ID ="prj-dtefpnc-p-bq-c3bc"
BQ_DATASET = "boost"
BQ_TABLE = "offres_v3"

cred_file = 'prj-dtefpnc-p-bq-c3bc-311cdf7d3088.json'
env_local = os.path.exists(cred_file)
if env_local:
    creds = service_account.Credentials.from_service_account_file(cred_file)
else:
    creds, _ = google.auth.default()

# Gestion des logs dans cloud run
logging_client = google.cloud.logging.Client()
logging_client.setup_logging()



In [29]:
BOOST_TABLE = "offre_emploi"
BOOST_URL = f"https://www.province-sud.nc/drhouseweb/api/GOUV_WEB/societe/{BOOST_TABLE}/data?apiKey={BOOST_TOKEN}"

response = requests.get(BOOST_URL)
response.json()

{'database': 'GOUV_WEB',
 'schema': 'societe',
 'table': 'offre_emploi',
 'hash': 'e1054ce99e6bc43b75c9ee3ec3b03bc0',
 'uris': {'dataUri': 'https://www.province-sud.nc/drhouseweb/api/GOUV_WEB/societe/offre_emploi/data',
  'swaggerUri': 'https://www.province-sud.nc/swagger/?url=/drhouseweb/api/GOUV_WEB/societe/offre_emploi/openapi',
  'openApiUri': 'https://www.province-sud.nc/drhouseweb/api/GOUV_WEB/societe/offre_emploi/openapi',
  'metadataUri': 'https://www.province-sud.nc/geocatweb/dcat/30cff11b-ba70-4017-905f-8e45ceb6ad4e'},
 'queryTimestamp': '2024-11-13 03:53:35',
 'paramsQuery': {},
 'hasNextPage': True,
 'paramsNextPageQuery': {'id|Gt': '2c9083a9687205e501687395b5f86394'},
 'data': [{'annee': None,
   'appellation': {'code': '10372', 'label': "Agent / Agente d'accueil"},
   'brevetEtCertificats': [],
   'commune': {'code': 'NOUMÉA', 'label': 'NOUMÉA'},
   'competence': [{'code': '100541', 'label': 'Techniques de communication'},
    {'code': '101949',
     'label': 'Techniques 

In [28]:
BOOST_TABLE = "olap_offre_emploi_log"
PARAMS = "&_operation_date|datetimeGt=2024-09-12&_order=_operation_date"
BOOST_URL = f"https://www.province-sud.nc/drhouseweb/api/GOUV_WEB/societe/{BOOST_TABLE}/data?apiKey={BOOST_TOKEN}{PARAMS}"

response = requests.get(BOOST_URL)
response.json()

{'database': 'GOUV_WEB',
 'schema': 'societe',
 'table': 'olap_offre_emploi_log',
 'hash': '4f94de6ccd2315b6f728298774855a51',
 'uris': {'dataUri': 'https://www.province-sud.nc/drhouseweb/api/GOUV_WEB/societe/olap_offre_emploi_log/data',
  'swaggerUri': 'https://www.province-sud.nc/swagger/?url=/drhouseweb/api/GOUV_WEB/societe/olap_offre_emploi_log/openapi',
  'openApiUri': 'https://www.province-sud.nc/drhouseweb/api/GOUV_WEB/societe/olap_offre_emploi_log/openapi',
  'metadataUri': 'https://www.province-sud.nc/geocatweb/dcat/30cff11b-ba70-4017-905f-8e45ceb6ad4e'},
 'queryTimestamp': '2024-11-13 03:53:10',
 'paramsQuery': {'_operation_date|datetimeGt': '2024-09-12',
  '_order': '_operation_date'},
 'hasNextPage': True,
 'paramsNextPageQuery': {'start': 50},
 'data': [{'_operation': 'INSERT',
   '_operation_date': '2024-09-12T16:44:47.534',
   'annee': None,
   'appellation': {'code': '10372', 'label': "Agent / Agente d'accueil"},
   'brevetEtCertificats': [],
   'commune': {'code': 'NOU

In [26]:

response.json()["paramsNextPageQuery"]["start"]

50

In [2]:
client = bigquery.Client(GCP_PROJECT_ID, credentials=creds,)

In [5]:
query = f"SELECT max(_operation_date) last_update FROM `{GCP_PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE}`"
query_job = client.query(query)

In [6]:
row.last_update

NameError: name 'row' is not defined

In [ ]:
def get_last_update_date(bq_client):
    try:
        query = f"SELECT max(_operation_date) last_update FROM `{GCP_PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE}`"
        query_job = bq_client.query(query)
        
        for row in query_job.result():
            last_update = row.last_update

        if last_update:
            if isinstance(last_update, str):
                # Si c'est une chaîne, convertir en objet datetime
                last_update = datetime.strptime(last_update, '%Y-%m-%dT%H:%M:%S.%f%z')
            
            # Convertir au format Jira
            jira_format_date = last_update.strftime('%Y/%m/%d %H:%M')
            logging.debug(f"Date formatée pour Jira: {jira_format_date}")
        else:
            logging.warning("Aucune date récupérée")
            return None

        return jira_format_date
    
    except Exception as e:
        logging.error(f"Une erreur s'est produite : {e}")
        
        return None

In [8]:

def normalize_commune(nom):
    if nom is not None:
        return unidecode(nom).lower()
    else:
        return None


In [27]:
BOOST_URL

'https://www.province-sud.nc/drhouseweb/api/GOUV_WEB/societe/olap_offre_emploi_log/data?apiKey=***REMOVED***&_operation_date|datetimeGt=2024-09-12&_order=_operation_date&start=0'

In [9]:
def fetch_offers(start=0):
    data = []

    response = requests.get(BOOST_URL+"&start={start}")
    response.raise_for_status()
    try:
        res = response.json()
    except ValueError as e:
        print(f"Erreur lors de l'extraction du JSON : {e}")
        print(f"Erreur de parse, reponse : {response.text}")

    data.extend(res["data"])
    while True:
        if "hasNextPage" in res and res["paramsNextPageQuery"] is not None:
            url = BOOST_URL+"&id%7CGt="+res["paramsNextPageQuery"]["start"]
            res = requests.get(url).json()
            data.extend(res["data"])
            for offre in res["data"]:
                if "logiciel" not in offre:
                    print(url)
            if len(data) % 5000 == 0:
                print(len(data))
        else:
            break
        
    return data



In [11]:
response = requests.get(BOOST_URL)
response.raise_for_status()

In [23]:
response = requests.get(BOOST_URL)
if response.status_code == 500 :
    print("ok")

In [12]:
response.status_code 

200

In [ ]:

def transform(offres):
    print("######## transform ########")
    for offre in offres:
        if isinstance(offre, dict):
            del offre["logiciel"]
        else:
            print(f"Error: Expected a dict but got : {type(offre)}")
            print(f"Offre : {offre}")
    return offres


def upload(json):
    print("######## upload ########")
    
    table_id = "%s.%s.%s" % (GCP_PROJECT_ID, BQ_DATASET, BQ_TABLE)
    table_ref = bigquery.Table(table_id)
    table = client.create_table(table_ref, exists_ok=True)
    job_config = bigquery.LoadJobConfig(autodetect = True,
                                        write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE)    
    job = client.load_table_from_json(json, table, job_config=job_config)
    job.result()
    print(f"Loaded {job.output_rows} rows into {table_id}")


In [ ]:

@functions_framework.http
def spe_boost(request):
    offres = load()
    print(len(offres))
    df = transform(offres)
    upload(df)
    return "transfert terminé"


if __name__ == "__main__":
    spe_boost(None)
